# Experiments of Different Model Arcitectures

This notebook runs all experiments for the final report. It uses the modular code from the `.py` modules to:
1.  Define multiple model configurations.
2.  Train and evaluate each configuration on multiple languages (English, Spanish, German).
3.  Collect and display dataset statistics (vocab size, tag count, etc.).
4.  Collate all performance metrics into summary tables.
5.  Run qualitative error analysis on a saved model.

## 1. Setup and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import keras
import os
import pickle
import json
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, f1_score

from data.preprocessor import DataPreprocessor, DataPreprocessorConfig
from models.base_model import ModelConfig
from models.lstm_model import LSTMModel
from trainer.trainer import TrainerConfig, Trainer
from evaluator.evaluator import Evaluator
from inference.predictor import Predictor
from utils import load_data

keras.utils.set_random_seed(50)

/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next r

## 2. Experiment Configuration

This is the main control panel. We can define all the languages and model architectures we want to test.

In [2]:
LANGUAGES = ["english", "spanish", "german"]

preprocessor_config = DataPreprocessorConfig(
    padding_type="post",
    truncation_type="post",
    remove_long_sentences=True,
    max_sequence_length=100, # As per your config
)

base_training_config = TrainerConfig(
    epochs=20,
    batch_size=64,
    early_stopping_patience=3,
    learning_rate=1e-3,
    model_dir="saved_models_experiment",
    save_best_only=True,
)


EXPERIMENT_CONFIGS = {
    "LSTM_Embed_80_LSTM_64": ModelConfig(
        embedding_dim=80,
        lstm_units=64,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "LSTM_Stacked_Embed_80_LSTM_64": ModelConfig(
        embedding_dim=80,
        lstm_units=64,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers = 2
    ),
    "LSTM_Embed_80_LSTM_128": ModelConfig(
        embedding_dim=80,
        lstm_units=128,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "LSTM_Stacked_Embed_80_LSTM_128": ModelConfig(
        embedding_dim=80,
        lstm_units=128,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers = 2
    ),
    "LSTM_Embed_128_LSTM_64": ModelConfig(
        embedding_dim=128,
        lstm_units=64,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "LSTM_Stacked_Embed_128_LSTM_64": ModelConfig(
        embedding_dim=128,
        lstm_units=64,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers = 2
    ),
    "LSTM_Embed_128_LSTM_128": ModelConfig(
        embedding_dim=128,
        lstm_units=128,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "LSTM_Stacked_Embed_128_LSTM_128": ModelConfig(
        embedding_dim=128,
        lstm_units=128,
        bidirectional=False,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers=2
    ),
    "BiLSTM_Embed_80_LSTM_64": ModelConfig(
        embedding_dim=80,
        lstm_units=64,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "BiLSTM_Stacked_Embed_80_LSTM_64": ModelConfig(
        embedding_dim=80,
        lstm_units=64,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers=2
    ),
    "BiLSTM_Embed_80_LSTM_128": ModelConfig(
        embedding_dim=80,
        lstm_units=128,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "BiLSTM_Stacked_Embed_80_LSTM_128": ModelConfig(
        embedding_dim=80,
        lstm_units=128,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers=2
    ),
    "BiLSTM_Embed_128_LSTM_64": ModelConfig(
        embedding_dim=128,
        lstm_units=64,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "BiLSTM_Stacked_Embed_128_LSTM_64": ModelConfig(
        embedding_dim=128,
        lstm_units=64,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers=2
    ),
    "BiLSTM_Embed_128_LSTM_128": ModelConfig(
        embedding_dim=128,
        lstm_units=128,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config
    ),
    "BiLSTM_Stacked_Embed_128_LSTM_128": ModelConfig(
        embedding_dim=128,
        lstm_units=128,
        bidirectional=True,
        dropout_rate=0.3,
        training_config=base_training_config,
        lstm_layers=2
    )
}

all_results = {}

if not os.path.exists(base_training_config.model_dir):
    os.makedirs(base_training_config.model_dir)

## 3. Main Experiment Loop

This loop will iterate through every model configuration and train/evaluate it on every language. All results will be saved.

In [3]:
for config_name, model_config in EXPERIMENT_CONFIGS.items():
    
    all_results[config_name] = {}
    
    for language in LANGUAGES:
        print(f"\n{'='*60}")
        print(f"RUNNING EXPERIMENT: Model='{config_name}', Language='{language}'")
        print(f"{'='*60}")
        
        all_results[config_name][language] = {}

        # 1. Load Data
        print("Loading data...")
        train_data, dev_data, test_data = load_data(language)
        all_results[config_name][language]['n_train_sents'] = len(train_data)
        all_results[config_name][language]['n_dev_sents'] = len(dev_data)
        all_results[config_name][language]['n_test_sents'] = len(test_data)

        # 2. Preprocess Data
        # We create a new preprocessor for each language to build language-specific vocabs
        print("Preprocessing data...")
        preprocessor = DataPreprocessor(preprocessor_config)

        X_train, y_train = preprocessor.process_data_to_pad_sequences(
            train_data, is_train_dataset=True
        )
        X_dev, y_dev = preprocessor.process_data_to_pad_sequences(
            dev_data, is_train_dataset=False
        )
        X_test, y_test = preprocessor.process_data_to_pad_sequences(
            test_data, is_train_dataset=False
        )
        
        # Save dataset stats for the report
        print(f"Vocab Size: {preprocessor.vocab_size}")
        print(f"Num Tags: {preprocessor.num_tags}")
        all_results[config_name][language]['vocab_size'] = preprocessor.vocab_size
        all_results[config_name][language]['num_tags'] = preprocessor.num_tags

        # 3. Initialize Model
        print("Initializing model...")
        model = LSTMModel(
            model_config,
            preprocessor.vocab_size,
            preprocessor.num_tags,
            preprocessor_config.max_sequence_length,
        )
        
        model.build_model()
        model.compile_model()
        print(model.get_model().summary()) # For trainable param count

        # 4. Train Model
        print("Training model...")
        trainer = Trainer(model_config.training_config, model, preprocessor)
        history = trainer.train((X_train, y_train), (X_dev, y_dev), language)
        print("Training completed.")
        
        # 5. Evaluate Model
        print("Evaluating model on test set...")
        # Note: The trainer auto-restores the best model. We use that for evaluation.
        evaluator = Evaluator(model, preprocessor)
        test_metrics = evaluator.evaluate(X_test, y_test, "Test")
        
        # Save key results
        all_results[config_name][language]['test_accuracy'] = test_metrics['accuracy']
        all_results[config_name][language]['test_f1_macro'] = f1_score(test_metrics['y_true'], test_metrics['y_pred'], average='macro')
        all_results[config_name][language]['test_f1_weighted'] = f1_score(test_metrics['y_true'], test_metrics['y_pred'], average='weighted')
        all_results[config_name][language]['classification_report'] = classification_report(test_metrics['y_true'], test_metrics['y_pred'], zero_division=0, output_dict=True)

        # 6. Save Model and Preprocessor for inference
        model_name = f"{config_name}_{language}"
        preprocessor_path = os.path.join(base_training_config.model_dir, f"{model_name}_preprocessor.pkl")
        
        with open(preprocessor_path, 'wb') as f:
            pickle.dump(preprocessor, f)
        
        print(f"--- Experiment for {config_name} on {language} FINISHED ---")

print("\n\nALL EXPERIMENTS COMPLETED.")

# Save results to a JSON file for persistence
results_path = os.path.join(base_training_config.model_dir, "all_experiment_results.json")
with open(results_path, 'w') as f:
    # We can't save the classification_report dict easily, let's simplify
    # A more robust way would be to flatten it, but this is fine for now.
    temp_results = all_results.copy()
    for config_name in temp_results:
        for language in temp_results[config_name]:
            if 'classification_report' in temp_results[config_name][language]:
                del temp_results[config_name][language]['classification_report']
                
    json.dump(temp_results, f, indent=4)

print(f"All results saved to {results_path}")


RUNNING EXPERIMENT: Model='LSTM_Embed_80_LSTM_64', Language='english'
Loading data...
Preprocessing data...
Vocab Size: 19676
Num Tags: 19
Initializing model...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 80)        │     1,574,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 64)        │        37,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 19)        │         1,235 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,612,435 (6.15 MB)

 Trainable params: 1,612,435 (6.15 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_94 (Embedding)        │ (None, 100, 128)       │     6,336,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_68                │ (None, 100, 256)       │       263,168 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_69                │ (None, 100, 256)       │       394,240 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_94             │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,998,419 (26.70 MB)

 Trainable params: 6,998,419 (26.70 MB)

 Non-trainable params: 0 (0.00 B)

None
Training model...
Epoch 1/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - _masked_accuracy: 0.3781 - loss: 1.9259
Epoch 1: val_loss improved from None to 0.47385, saving model to saved_models_experiment/BiLSTM_Stacked_Embed128_LSTM128_german.keras
216/216 ━━━━━━━━━━━━━━━━━━━━ 84s 367ms/step - _masked_accuracy: 0.5912 - loss: 1.2467 - val__masked_accuracy: 0.8678 - val_loss: 0.4738
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/step - _masked_accuracy: 0.9001 - loss: 0.3314
Epoch 2: val_loss improved from 0.47385 to 0.30636, saving model to saved_models_experiment/BiLSTM_Stacked_Embed128_LSTM128_german.keras
216/216 ━━━━━━━━━━━━━━━━━━━━ 98s 454ms/step - _masked_accuracy: 0.9259 - loss: 0.2560 - val__masked_accuracy: 0.9099 - val_loss: 0.3064
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - _masked_accuracy: 0.9638 - loss: 0.1342
Epoch 3: val_loss improved from 0.30636 to 0.29155, saving model to saved_models_experiment/BiLSTM_Stacked_Embed128_LSTM128_german.keras
216/216 ━

## 4. Collate and Display Results

This section formats the collected data from the `all_results` dictionary into markdown tables, ready to be copied into your report.

In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os

# Set seaborn style for better-looking plots
sns.set_theme(style="whitegrid")

# --- 1. Load and Flatten Data ---
print("Loading all_experiment_results.json...")
# Ensure the file is in the same directory as this notebook
with open('saved_models_experiment/all_experiment_results.json', 'r') as f:
    data = json.load(f)

# Flatten the nested JSON into a list of dictionaries
flat_data = []
for model_name, languages in data.items():
    for language, metrics in languages.items():
        # Create a new record for each model-language pair
        record = {
            'model': model_name,
            'language': language,
            **metrics  # Add all metric key-value pairs
        }
        flat_data.append(record)

# Convert to Pandas DataFrame
df = pd.DataFrame(flat_data)

# Feature Engineering
# Extract key hyperparameters from the model name
df['Direction'] = df['model'].apply(lambda x: 'BiLSTM' if 'BiLSTM' in x else 'LSTM')
df['Embedding Dim'] = df['model'].apply(lambda x: 128 if 'Embed_128' in x else 80).astype(int)
df['LSTM Units'] = df['model'].apply(lambda x: 128 if 'LSTM_128' in x else 64).astype(int)
df['Stacked'] = df['model'].apply(lambda x: 'Stacked' if 'Stacked' in x else 'Single')


# Create a cleaner label for plotting
df['Model Label'] = df['model'].str.replace('_', ' ').str.replace('LSTM', ' LSTM')

# Directory to save plots
plot_dir = "seaborn_plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
print(f"Plots will be saved to '{plot_dir}/'")

# --- 3. Generate Plots ---
plot_files = []

# Plot 1: Dataset Statistics (Vocabulary Size)
print("Generating Plot 1: Vocabulary Size by Language...")
plt.figure(figsize=(8, 5))
# We only need one entry per language, so we drop duplicates
df_stats = df.drop_duplicates(subset='language')
sns.barplot(data=df_stats, x='language', y='vocab_size', palette='viridis')
plt.title('Vocabulary Size by Language', fontsize=16)
plt.ylabel('Vocabulary Count', fontsize=12)
plt.xlabel('Language', fontsize=12)
plot_path = os.path.join(plot_dir, '1_vocab_size_by_language.png')
plt.savefig(plot_path)
plt.close()

# Plot 2: Key Insight - BiLSTM vs. Unidirectional LSTM
print("Generating Plot 2: Directionality Comparison...")
g = sns.catplot(
    data=df, 
    x='language', 
    y='test_accuracy', 
    hue='Direction', 
    kind='bar', 
    palette='colorblind',
    aspect=1.5,
    legend_out=False
)
g.fig.suptitle('BiLSTM vs. Unidirectional LSTM (Test Accuracy)', fontsize=16, y=1.03)
g.set_axis_labels('Language', 'Test Accuracy', fontsize=12)
plt.ylim(0.85, 0.95) # Emphasize the difference
g.ax.legend(title='Direction', loc='upper right')
plot_path = os.path.join(plot_dir, '2_directionality_comparison.png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()


Loading all_experiment_results.json...
Plots will be saved to 'seaborn_plots/'
Generating Plot 1: Vocabulary Size by Language...
Generating Plot 2: Directionality Comparison...
Generating Plot 3: Accuracy vs. F1-Macro...


/tmp/ipykernel_81105/1076339606.py:56: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_stats, x='language', y='vocab_size', palette='viridis')


In [ ]:
# Plot 3: Accuracy vs. F1-Macro
# Find the best model overall (highest mean F1-Macro)
best_model_name = df.groupby('model')['test_f1_macro'].mean().idxmax()
df_best_model = df[df['model'] == best_model_name]

# Melt the dataframe to plot multiple metrics
df_melted = df_best_model.melt(
    id_vars='language', 
    value_vars=['test_accuracy', 'test_f1_macro'], 
    var_name='Metric', 
    value_name='Score'
)
# Make labels cleaner
df_melted['Metric'] = df_melted['Metric'].map({
    'test_accuracy': 'Accuracy',
    'test_f1_macro': 'F1-Macro'
})

g = sns.catplot(
    data=df_melted, 
    x='language', 
    y='Score', 
    hue='Metric', 
    kind='bar', 
    palette='Pastel1',
    aspect=1.5
)
g.fig.suptitle(f'Accuracy vs. F1-Macro (Best Model: {best_model_name})', fontsize=16, y=1.03)
g.set_axis_labels('Language', 'Score', fontsize=12)
plt.ylim(0.7, 1.0)
plot_path = os.path.join(plot_dir, '3_accuracy_vs_f1_macro.png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

In [23]:
# Plots 4: Model Complexity Heatmaps
df_eng_bilstm = df[(df['language'] == 'english') & (df['Direction'] == 'BiLSTM') & (df['Stacked'] == 'Single')]
pivot_table = df_eng_bilstm.pivot_table(
    index='Embedding Dim', 
    columns='LSTM Units', 
    values='test_accuracy'
)

plt.figure(figsize=(7, 5))
sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap="viridis", linewidths=.5)
plt.title('English BiLSTM: Test Accuracy Heatmap', fontsize=16)
plt.xlabel('LSTM Hidden Units', fontsize=12)
plt.ylabel('Embedding Dimension', fontsize=12)
plot_path = os.path.join(plot_dir, '4_english_bilstm_heatmap.png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

# ----------------------------------------

df_es_bilstm = df[(df['language'] == 'spanish') & (df['Direction'] == 'BiLSTM') & (df['Stacked'] == 'Single')]
pivot_table = df_es_bilstm.pivot_table(
    index='Embedding Dim', 
    columns='LSTM Units', 
    values='test_accuracy'
)

plt.figure(figsize=(7, 5))
sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap="viridis", linewidths=.5)
plt.title('Spanish BiLSTM: Test Accuracy Heatmap', fontsize=16)
plt.xlabel('LSTM Hidden Units', fontsize=12)
plt.ylabel('Embedding Dimension', fontsize=12)
plot_path = os.path.join(plot_dir, '4_spanish_bilstm_heatmap.png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

# ----------------------------------------

df_ge_bilstm = df[(df['language'] == 'german') & (df['Direction'] == 'BiLSTM') & (df['Stacked'] == 'Single')]
pivot_table = df_ge_bilstm.pivot_table(
    index='Embedding Dim', 
    columns='LSTM Units', 
    values='test_accuracy'
)

plt.figure(figsize=(7, 5))
sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap="viridis", linewidths=.5)
plt.title('German BiLSTM: Test Accuracy Heatmap', fontsize=16)
plt.xlabel('LSTM Hidden Units', fontsize=12)
plt.ylabel('Embedding Dimension', fontsize=12)
plot_path = os.path.join(plot_dir, '4_german_bilstm_heatmap.png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

In [ ]:
# Plot 5: Full Model Comparison (All Models, All Languages)
df_sorted = df.sort_values(by='test_accuracy', ascending=False)
plt.figure(figsize=(12, 10))
sns.barplot(
    data=df_sorted, 
    x='test_accuracy', 
    y='Model Label', 
    hue='language', 
    palette='muted'
)
plt.title('Overall Model Performance Comparison (Test Accuracy)', fontsize=16)
plt.xlabel('Test Accuracy', fontsize=12)
plt.ylabel('Model Configuration', fontsize=12)
plt.legend(title='Language', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.axvline(x=0.9, color='r', linestyle='--', label='90% Accuracy') # Add a reference line
plt.xlim(0.85, 0.95)
plot_path = os.path.join(plot_dir, '5_full_model_comparison(acc).png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

In [16]:
# Plot 6: Full Model Comparison (All Models, All Languages)
df_sorted = df.sort_values(by='test_f1_macro', ascending=False)
plt.figure(figsize=(12, 10))
sns.barplot(
    data=df_sorted, 
    x='test_f1_macro', 
    y='Model Label', 
    hue='language', 
    palette='muted'
)
plt.title('Overall Model Performance Comparison (Macro F1)', fontsize=16)
plt.xlabel('Macro F1', fontsize=12)
plt.ylabel('Model Configuration', fontsize=12)
plt.legend(title='Language', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xlim(0.7, 0.9)
plot_path = os.path.join(plot_dir, '6_full_model_comparison(f1).png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

In [17]:
# Plot 7: Full Model Comparison (All Models, All Languages)
df_sorted = df.sort_values(by='test_f1_weighted', ascending=False)
plt.figure(figsize=(12, 10))
sns.barplot(
    data=df_sorted, 
    x='test_f1_weighted', 
    y='Model Label', 
    hue='language', 
    palette='muted'
)
plt.title('Overall Model Performance Comparison (Weighted F1)', fontsize=16)
plt.xlabel('Weighted F1', fontsize=12)
plt.ylabel('Model Configuration', fontsize=12)
plt.legend(title='Language', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xlim(0.8, 0.95)
plot_path = os.path.join(plot_dir, '7_full_model_comparison(f1_weighted).png')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

In [20]:
g = sns.catplot(
    data=df, 
    x='language', 
    y='test_accuracy', 
    hue='Stacked', 
    kind='bar', 
    palette='colorblind',
    aspect=1.5,
    legend_out=False
)
g.fig.suptitle('Stacked vs. Single LSTM (Test Accuracy)', fontsize=16, y=1.03)
g.set_axis_labels('Language', 'Test Accuracy', fontsize=12)
plt.ylim(0.85, 0.95) # Emphasize the difference
g.ax.legend(title='Stacked', loc='upper right')
plot_path = os.path.join(plot_dir, '8_stacked_comparison.png')
plt.savefig(plot_path, bbox_inches='tight')
plot_files.append(plot_path)
plt.close()

## 5. Qualitative Error Analysis

Here, we load our best-performing model (e.g., `BiLSTM_Embed_128` for English) and its corresponding preprocessor to run inference on new, custom sentences. This allows us to find specific examples of 'good', 'bad', and 'ugly' predictions for the report.

In [5]:
# --- 1. Load the Model and Preprocessor ---

# Define which saved model we want to test
TARGET_CONFIG = "BiLSTM_Embed_128_LSTM_128"
TARGET_LANGUAGE = "english"

model_name = f"{TARGET_CONFIG}_{TARGET_LANGUAGE}"
model_path = os.path.join(base_training_config.model_dir, f"{model_name}.keras")
preprocessor_path = os.path.join(base_training_config.model_dir, f"{model_name}_preprocessor.pkl")

print(f"Loading model from: {model_path}")

try:
    loaded_model = keras.models.load_model(model_path)
    
    print(f"Loading preprocessor from: {preprocessor_path}")
    with open(preprocessor_path, 'rb') as f:
        loaded_preprocessor = pickle.load(f)
        
    # 2. Create Predictor Instance
    predictor = Predictor(loaded_model, loaded_preprocessor)

    print("\n--- Predictor Ready. Running Qualitative Tests. ---\n")
    
    # --- 3. Run Test Cases ---
    
    test_sentences = {
        "The 'Good' (Simple Case)": [
            "Today it is cloudy",
            "The quick brown fox jumps over the lazy dog ."
        ],
        "The 'Bad' (Ambiguity)": [
            "The leaves are falling .", # leaves = NOUN
            "He leaves tomorrow .", # leaves = VERB
            "I bought an apple .", # apple = NOUN
            "I work at Apple ." # Apple = PROPN
        ],
        "The 'Ugly' (OOV & Typos)": [
            "I googled this supercalifragilisticexpialidocious wrd .",
            "This sentance has twoo mispellings ."
        ]
    }

    for category, sentences in test_sentences.items():
        print(f"\n--- {category} ---")
        for sentence in sentences:
            predicted_tags = predictor.predict_sentence(sentence)
            print(f"  Sentence: {sentence}")
            print(f"  Tags:     {' '.join(predicted_tags)}")
            # Optional: Print word-tag pairs
            # print(f"  Result:   {list(zip(sentence.split(), predicted_tags))}")

except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}")
    print("Please ensure the main experiment loop (Section 3) has been run successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


Loading model from: saved_models_experiment/BiLSTM_Embed_128_LSTM_128_english.keras
An error occurred: Could not locate class 'method'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'builtins', 'class_name': 'method', 'config': '_masked_accuracy', 'registered_name': 'method'}
